                                                            M2-AMSS-CSC
<div style="text-align: center;">
    <img src="./logo_ens.png">
</div>

<div style="text-align: center;">
    <H1>Techniques et outils pour la preuve de concepts</H1>
    Alberic Junior DASSI FEUSSOUO
</div>

 <H3><i> Machine learning-based approach for online fault Diagnosis of Discrete Event System (Saddem et al. - 2022 )</H3> 

# 2. Acquisition des donnees
Les données utilisées dans l'article ont été recueillies sur un jumeau numérique grâce à une application développée en JavaScript avec le logiciel Node-RED (<i>paragraphe 3.4</i>). La difficulté d'accès à ces dernières nous a amené à développer un simulateur de SED simplifié, constitué de 2 capteurs binaires nommés "capteur_1" et "capteur_2", et d'un actionneur que nous avons appelé "moteur". Il s'agit d'un convoyeur. L'objectif c'est de générer les données qui vont nous permettre d'entrainer notre modèle de LSTM-RNN.


# <H3> 2.1. Bibliothèques nécessaires 

Le livrable attendu est un fichier Excel ou CSV contenant les données nécessaires pour entrainer l’algorithme. Nous aurons alors besoin des bibliothèques permettant la manipulation des données et enregistrement au format requis (panda), la génération de nombres aléatoires(random) et l'horodatage(datetime).

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import random

# <H3> 2.2. Modèle du SED

Le système que nous allons modéliser : 
* Est constitué d'un moteur et de deux capteurs délivrant des signaux binaires.
* Peut se trouver dans un <b>état normal</b> (C0) ou dans un <b>état défaillant</b> (C1 a C6 pour les défauts de blocage spécifiques des composants et C7 pour une transition inattendue) <br>
De plus, pour générer nos données, nous devons considérer deux cycles de fonctionnement que sont : <b>cycle normal et cycle défaillant</b>. Le second ne peut être observé qu'après <b>l'injection des défauts</b> (Paragraphe 3.4 de l'article). Enfin, les données doivent être <b>sauvegardées dans un fichier CSV</b> pour exploitation ultérieure (<a  href ="./preparation_des_donnees.ipynb"><i>dans la section portant préparation des données</i></a>).

In [2]:
class SAP:
    """
    Simulateur de Systeme Automatise de Production
    """
    
    def __init__(self): 
        # Configuration du système/Construction des composants du systeme
        self.components = { 
            'capteur_1': 0,    # Capteur au début du convoyeur (0 s'il est inactif)
            'capteur_2': 0,   # Capteur a la fin du convoyeur
            'moteur': 0,    # Moteur du convoyeur 0=> arreté, (1 lorsqu'il est en marche)
        }
        
        # Les états du systeme
        self.states = {
            'C0': 'Normal', #définit le fonctionnement normal du systeme
            'C1': 'capteur_1_Bloque_sur_0',
            'C2': 'capteur_1_Bloque_sur_1',
            'C3': 'capteur_2_Bloque_sur_0',
            'C4': 'capteur_2_Bloque_sur_1',
            'C5': 'moteur_Bloque_sur_0',
            'C6': 'moteur_Bloque_sur_1',
            'C7': 'transition_inattendue'  #défauts de transitions inattendues (0 vers 1 ou 1 vers 0)
        }
        
        self.current_state = 'C0' #Etat actuel du systeme (par défaut C0)
        self.fault_active = None  #Pour indiquer si un defaut est actif (C1,C2,...)
        self.fault_injected_at = None
        self.history = [] #Liste pour enregistrer l'historique des états

    # Comportement durant un cycle normal de fonctionnement
    # la logique de fonctionnement normal est : Objet - capteur_1 - moteur - capteur_2 
    #duration_steps: Durée totale de simulation
    def normal_cycle(self, duration_steps=100):
        
        data = []
        start_time = datetime.now()
        for step in range(duration_steps):
            timestamp = start_time + timedelta(seconds=step*0.1)
            
            if step % 20 == 0:  # Nouvelle pièce tous les 20 steps
                self.components['capteur_1'] = 1
            elif step % 20 == 2:
                self.components['capteur_1'] = 0
                self.components['moteur'] = 1
            elif step % 20 == 15:
                self.components['capteur_2'] = 1
            elif step % 20 == 17:
                self.components['capteur_2'] = 0
                self.components['moteur'] = 0
            
            # Enregistrer l'état
            data.append({
                'timestamp': timestamp,
                'capteur_1': self.components['capteur_1'],
                'capteur_2': self.components['capteur_2'],
                'moteur': self.components['moteur'],
                'state': self.current_state,
                'state_label': self.states[self.current_state]
            })
        
        return pd.DataFrame(data)

    # Simulation d'un cycle avec defaut
    #fault_type: Type de défaut (C1 à C7)
    #duration_steps: Durée totale de simulation
    #fault_start: Step où le défaut apparaît
    
    def faulty_cycle(self, fault_type, duration_steps=100, fault_start=30):
      
        data = []
        start_time = datetime.now()
        
        for step in range(duration_steps):
            timestamp = start_time + timedelta(seconds=step*0.1)
            
            # Injection de défaut au moment spécifié
            if step == fault_start:
                self.current_state = fault_type
                self.fault_active = fault_type
                self.fault_injected_at = step
            
            # Comportement normal jusqu'au défaut
            if step < fault_start or self.fault_active is None:
                # Cycle normal
                if step % 20 == 0:
                    self.components['capteur_1'] = 1
                elif step % 20 == 2:
                    self.components['capteur_1'] = 0
                    self.components['moteur'] = 1
                elif step % 20 == 15:
                    self.components['capteur_2'] = 1
                elif step % 20 == 17:
                    self.components['capteur_2'] = 0
                    self.components['moteur'] = 0
            else:
                # Application du défaut
                self._apply_fault(step, fault_start)
            
            # Enregistrer l'état
            data.append({
                'timestamp': timestamp,
                'capteur_1': self.components['capteur_1'],
                'capteur_2': self.components['capteur_2'],
                'moteur': self.components['moteur'],
                'state': self.current_state,
                'state_label': self.states[self.current_state]
            })
        
        return pd.DataFrame(data)
        
    #Simulation du comportement defaillant selon le type de defaut
    
    def _apply_fault(self, step, fault_start):
        relative_step = (step - fault_start) % 20
        
        # Defaut de blocage
        
        if self.fault_active == 'C1':  # capteur_1 bloqué à 0
            self.components['capteur_1'] = 0
            if relative_step == 2:
                self.components['moteur'] = 0 # Le moteur reste inactif car capteur_1 est bloqué sur 0
            elif relative_step == 15:
                self.components['capteur_2'] = 1
            elif relative_step == 17:
                self.components['capteur_2'] = 0
                #self.components['moteur'] = 0
                
        elif self.fault_active == 'C2':  # capteur_1 bloqué à 1
            self.components['capteur_1'] = 1
            if relative_step == 2:
                self.components['moteur'] = 1
            elif relative_step == 15:
                self.components['capteur_2'] = 1
            elif relative_step == 17:
                self.components['capteur_2'] = 0
                self.components['moteur'] = 0
                
        elif self.fault_active == 'C3':  # capteur_2 bloqué à 0
            self.components['capteur_2'] = 0
            if relative_step == 0:
                self.components['capteur_1'] = 1
            elif relative_step == 2:
                self.components['capteur_1'] = 0
                self.components['moteur'] = 1
            elif relative_step == 17:
                self.components['moteur'] = 0
                
        elif self.fault_active == 'C4':  # capteur_2 bloqué à 1
            self.components['capteur_2'] = 1
            if relative_step == 0:
                self.components['capteur_1'] = 1
            elif relative_step == 2:
                self.components['capteur_1'] = 0
                self.components['moteur'] = 1
            elif relative_step == 17:
                self.components['moteur'] = 0
                
        elif self.fault_active == 'C5':  # Moteur bloqué à 0
            self.components['actuator_motor'] = 0
            if relative_step == 0:
                self.components['capteur_1'] = 1
            elif relative_step == 2:
                self.components['capteur_1'] = 0
            # Le moteur ne démarre jamais
                
        elif self.fault_active == 'C6':  # Moteur bloqué à 1
            self.components['moteur'] = 1
            if relative_step == 0:
                self.components['capteur_1'] = 1
            elif relative_step == 2:
                self.components['capteur_1'] = 0
            elif relative_step == 15:
                self.components['capteur_2'] = 1
            elif relative_step == 17:
                self.components['capteur_2'] = 0
            # Moteur reste toujours à 1
        
        # C7: transitions inattendues
        # On regroupe tous les types de transitions inattendues (0-1 ou 1-0)
        
        elif self.fault_active == 'C7':
            # cas 1: capteur_1 passe à 1 sans raison
            if relative_step == 5:
                self.components['capteur_1'] = 1  # Transition 0-1 inattendue
            elif relative_step == 7:
                self.components['capteur_1'] = 0
            
            # cas 2: Moteur s'arrête sans raison
            if relative_step == 0:
                self.components['capteur_1'] = 1
            elif relative_step == 2:
                self.components['capteur_1'] = 0
                self.components['moteur'] = 1
            elif relative_step == 8:
                self.components['moteur'] = 0  # Transition 1-0 inattendue
            elif relative_step == 10:
                self.components['moteur'] = 1  # Redémarre
            elif relative_step == 15:
                self.components['capteur_2'] = 1
            elif relative_step == 17:
                self.components['capteur_2'] = 0
                self.components['moteur'] = 0
   
    # fonction de generation de jeu de données complet avec données normales et défaillantes
    # n_normal: Nombre de cycles normaux
    # n_faulty_per_type: Nombre de cycles par type de défaut
    
    def generate_dataset(self, n_normal=500, n_faulty_per_type=100):
        all_data = []
        
        print("Génération des données normales...")
        # Données normales
        for i in range(n_normal // 100):
            self.__init__()  # Reset
            df_normal = self.normal_cycle(duration_steps=100)
            all_data.append(df_normal)
        
        #print("Génération des données avec défauts...")
        # Données avec défauts (C1 à C7)
        fault_types = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
        
        for fault in fault_types:
            for i in range(n_faulty_per_type // 100):
                self.__init__()  # Reset
                fault_start = random.randint(20, 40)  # Défaut apparaît aléatoirement
                df_faulty = self.faulty_cycle(fault, duration_steps=100, fault_start=fault_start)
                all_data.append(df_faulty)
        
        # Combinaison de toutes les données
        full_dataset = pd.concat(all_data, ignore_index=True)
        
        print(f"\nJeu de données généré: {len(full_dataset)} échantillons")
        print("\nDistribution des états:")
        print(full_dataset['state'].value_counts())
        
        return full_dataset

# <H3> 2.3 Création d’un simulateur 

Le modèle étant défini, nous passons à la création d'un SED qui nous fournira des données en sortie, dans un fichier CSV.

In [4]:
if __name__ == "__main__":
    print("="*60)
    print("SIMULATEUR")
    print("="*60)
    
    # On crée le simulateur
    simulator = SAP()
    
    # 1. Généreration d'un scénario normal
    print("\n1. Génération d'un scénario normal...")
    df_normal = simulator.normal_cycle(duration_steps=100)
    print(df_normal.head(20))
    
    
    # 2. Généreration des scénarios avec défauts
    #print("\n2. Génération de scénarios avec DÉFAUTS...")
    
    # Test des différents types de défauts
    faults_to_test = ['C1', 'C3', 'C5', 'C7']
    for fault in faults_to_test:
        simulator = SAP()
        df_faulty = simulator.faulty_cycle(fault, duration_steps=100, fault_start=30)
    
    # 3. Généreration du jeu de donees complet
    #print("\ Génération du jeu de données COMPLET...")
    simulator = SAP()
    dataset = simulator.generate_dataset(n_normal=500, n_faulty_per_type=100)
    
    # Sauvegarder
    dataset.to_csv('sap_dataset.csv', index=False)
    print("\n Jeu de données sauvegardé dans 'sap_dataset.csv'")
    
    # Statistiques
    print("\n" + "="*60)
    print("STATISTIQUES DU JEU DE DONNEES")
    print("="*60)
    print(f"Nombre total d'échantillons: {len(dataset)}")
    print(f"\nDistribution par état:")
    print(dataset['state'].value_counts().sort_index())
    print(f"\nDistribution par label:")
    print(dataset['state_label'].value_counts())

SIMULATEUR

1. Génération d'un scénario normal...
                    timestamp  capteur_1  capteur_2  moteur state state_label
0  2025-12-29 11:08:31.449730          1          0       0    C0      Normal
1  2025-12-29 11:08:31.549730          1          0       0    C0      Normal
2  2025-12-29 11:08:31.649730          0          0       1    C0      Normal
3  2025-12-29 11:08:31.749730          0          0       1    C0      Normal
4  2025-12-29 11:08:31.849730          0          0       1    C0      Normal
5  2025-12-29 11:08:31.949730          0          0       1    C0      Normal
6  2025-12-29 11:08:32.049730          0          0       1    C0      Normal
7  2025-12-29 11:08:32.149730          0          0       1    C0      Normal
8  2025-12-29 11:08:32.249730          0          0       1    C0      Normal
9  2025-12-29 11:08:32.349730          0          0       1    C0      Normal
10 2025-12-29 11:08:32.449730          0          0       1    C0      Normal
11 2025-12-29 